<a href="https://colab.research.google.com/github/tvmnhajat/ID3-play-tennis/blob/main/ID3_play_tennis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd

In [9]:
class Node:

    def __init__(self, attribute, threshold):
        self.attribute = attribute
        self.threshold = threshold
        self.frame = None
        self.children = []
        self.leaf = False
        self.label = None

    def add_frame(self, frame):
        self.frame = frame

In [10]:
def entropy(data: pd.DataFrame, label):
    all = data[label].count()
    negative = data[data[label] == 0][label].count()
    positive = all - negative
    negative = negative + 0.00000001
    positive = positive + 0.00000001
    all = all + 0.00000001
    return -((negative/all * np.log2(negative/all)) + (positive/all * np.log2(positive/all)))

In [11]:
def gain(data: pd.DataFrame, label):
    list_features = data.nunique()
    features = list_features.keys()
    ret = {}

    curr_entropy = entropy(data, label)

    for i in features:
        if i == label: break
        feature_count = data[i].value_counts()
        feature_count = feature_count / feature_count.values.sum()
        a = [data[data[i] == j] for j in feature_count.keys()]
        temporary_entropy_i = np.array([entropy(a_item, label) for a_item in a])
        ret[i] = (curr_entropy - (np.array(feature_count.values) * temporary_entropy_i).sum())

    return ret

In [12]:
def decision_tree_algorithm(root, data: pd.DataFrame, label):
    '''
    Với root là nút gốc (tức nút đang được phân chia),
    I là tập các đặc trưng,
    O là tập nhãn
    T là tập huấn luyện
    '''
    I = data.columns
    O = [1, 0]

    if (len(data) == 0): # T trống, trả về nút có kết quả thất bại
        root.label = "Failure"
        return root

    elif (len(set(data[label])) == 1):
        root.frame = data
        root.leaf = True
        root.label = data[label][0]
        return root

    elif (data.columns < 2):
        root.leaf = True
        root.label = data["Play Tennis"].value_counts().idxmax()
        return root

    else:

        tmp_gain = gain(data, label)
        max_gain_attr = max(tmp_gain, key = tmp_gain.get)